In [5]:
%matplotlib inline

In [8]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [15]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


In [17]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [18]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [20]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3036,  0.2959,  0.1785,  0.7729,  0.5227,  0.6953, -0.1261, -0.4670,
         -0.4554,  0.1304,  0.3709, -0.5019, -0.6033, -0.2733, -0.0394, -0.3012,
          0.3553,  0.1499, -0.3242, -0.1261],
        [ 0.4177,  0.0931, -0.2966,  0.3874,  0.4775,  0.7979,  0.4700, -0.2914,
         -0.2002,  0.0719,  0.5401, -0.8229, -0.5752, -0.3548,  0.3424, -0.5014,
         -0.3292,  0.1791, -0.0112, -0.1456],
        [ 0.0571,  0.0415, -0.1277,  0.3296,  0.4829,  0.6523,  0.4473, -0.4245,
         -0.5544,  0.1202,  0.5672, -0.4657, -0.1365, -0.3403, -0.0585, -0.1551,
         -0.0409,  0.4201, -0.5398, -0.0849]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2959, 0.1785, 0.7729, 0.5227, 0.6953, 0.0000, 0.0000, 0.0000,
         0.1304, 0.3709, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3553, 0.1499,
         0.0000, 0.0000],
        [0.4177, 0.0931, 0.0000, 0.3874, 0.4775, 0.7979, 0.4700, 0.0000, 0.0000,
         0.0719, 0.5401, 0.0000, 0.0000, 0.0000, 0.34

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [23]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0181, -0.0141,  0.0096,  ..., -0.0220, -0.0210, -0.0161],
        [-0.0343, -0.0081,  0.0133,  ...,  0.0171, -0.0310, -0.0285]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0299,  0.0005], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0336, -0.0057,  0.0369,  ..., -0.0212, -0.0352, -0.0019],
        [-0.0017,  0.0121,  0.0079,  ..., -0.0124,  0.0038, -0.0044]],
       device='cu